In [4]:
%load_ext autoreload
%autoreload 2
from embedder import Embedder
import numpy as np
import pandas as pd


In [5]:
import tqdm
import json

embedder = Embedder()
data = json.load(open('data/wikidata.json'))
embeddings = {}


In [8]:
tokens = {concept:embedder.tokenize(data[concept]) for concept in data}


In [12]:
for i, concept in enumerate(tokens):
    if len(tokens[concept]) > 5000:
        print (concept,'\t\t\t\t', len(tokens[concept]))
        print (data[concept])
        print ()
        

https://en.wikipedia.org/wiki/Normal_distribution#Cumulative_distribution_function 				 9036
b'The cumulative distribution function (CDF) of the standard normal distribution, usually denoted with the capital Greek letter  (phi), is the integral\n\n\nThe related error function  gives the probability of a random variable with normal distribution of mean 0 and variance 1/2 falling in the range ; that is\n\n\nThese integrals cannot be expressed in terms of elementary functions, and are often said to be special functions. However, many numerical approximations are known; see below.\nThe two functions are closely related, namely\n\n\nFor a generic normal distribution with density , mean  and deviation , the cumulative distribution function is\n\n\nThe complement of the standard normal CDF, , is often called the Q-function, especially in engineering texts. It gives the probability that the value of a standard normal random variable  will exceed : . Other definitions of the -function, all of w

In [6]:
for concept in tqdm.tqdm(data):
    embeddings['concept'] = embedder.embed(embedder.tokenize(data[concept]))


  0%|                                                                                         | 0/1034 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0228 18:59:07.423355 17812 deprecation_wrapper.py:119] From C:\Users\spark\PycharmProjects\capstone\embedder\embedder.py:22: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0228 18:59:08.904329 17812 deprecation_wrapper.py:119] From C:\Users\spark\PycharmProjects\capstone\embedder\embedder.py:23: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0228 18:59:09.524894 17812 deprecation_wrapper.py:119] From C:\Users\spark\PycharmProjects\capstone\embedder\embedder.py:24: The name tf.tables_initializer is deprecated. Please use tf.compat.v1.tables_initializer instead.

  6%|████▉                                                                         | 65/1034 [08:01<3:41:45, 13.73s/it]

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[9036,256,1,46] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node module_apply_default_65/bilm/CNN/Conv2D_4 (defined at c:\users\spark\tfenv\lib\site-packages\tensorflow_hub\native_module.py:610) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[module_apply_default_65/truediv/_157]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[9036,256,1,46] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node module_apply_default_65/bilm/CNN/Conv2D_4 (defined at c:\users\spark\tfenv\lib\site-packages\tensorflow_hub\native_module

In [2]:
embedder

NameError: name 'embedder' is not defined